In [1]:
import os

In [2]:
%pwd

'/mnt/f/DS_Tech_Stack/MLOps/Chest-Cancer-Classification-using-MLflow-DVC/research'

In [3]:
os.chdir('../')

In [19]:
%pwd

'/mnt/f/DS_Tech_Stack/MLOps/Chest-Cancer-Classification-using-MLflow-DVC'

In [29]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/faridkhan5/Chest-Cancer-Classification-using-MLflow-DVC.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'faridkhan5'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '801f7f4defd33f302dd86616acda87ee94a5d725'

In [30]:
import tensorflow as tf

In [31]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

## Entity

In [32]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

## Config

In [33]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [34]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(path_of_model='artifacts/training/model.h5',
                                       training_data='artifacts/data_ingestion/Chest-CT-Scan-data',
                                       mlflow_uri='https://dagshub.com/faridkhan5/Chest-Cancer-Classification-using-MLflow-DVC.mlflow',
                                       all_params=self.params,
                                       params_image_size=self.params.IMAGE_SIZE,
                                       params_batch_size=self.params.BATCH_SIZE)
        return eval_config

## Components

In [35]:
import urllib.request as request
from urllib.parse import urlparse
from zipfile import ZipFile
import tensorflow as tf
import time
from cnnClassifier.utils.common import save_json
import mlflow

In [36]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1./255,
                                    validation_split=0.3)
        
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1],
                               batch_size=self.config.params_batch_size,
                               interpolation='bilinear')    

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        #create a datagenerator that loads data from the specified dir for validation
        self.valid_generator = valid_datagenerator.flow_from_directory(directory=self.config.training_data,
                                                                       subset='validation',
                                                                       shuffle=False,
                                                                       **dataflow_kwargs)
        
    
    @staticmethod
    def load_model(path: Path):
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    
    def save_score(self):
        scores = {'loss': self.score[0],
                  'accuracy': self.score[1]}
        save_json(path=Path('scores.json'), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                'loss': self.score[0],
                'accuracy': self.score[1]
            })
            
            #model registry doesn't work with file store
            if tracking_url_type_store != 'file':
                #register the model
                #there are other ways to use the model registry, whjich depends on the use case
                mlflow.keras.log_model(self.model, 'model', registered_model_name='VGG16Model')
            else:
                mlflow.keras.log_model(self.model, 'model')

## Pipeline

In [37]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-17 19:08:49,128: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-17 19:08:49,140: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-17 19:08:49,146: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 19s 2s/step - loss: 31.7146 - accuracy: 0.4314
[2024-04-17 19:09:09,861: INFO: common: json file saved at: scores.json]


2024/04/17 19:09:11 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmp082wt9tf/model/data/model/assets
[2024-04-17 19:09:12,764: INFO: builder_impl: Assets written to: /tmp/tmp082wt9tf/model/data/model/assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/04/17 19:09:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
